In [1]:
import os
import sys
import time
import torch
import numpy as np
from glob import glob
from PIL import Image

import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

from torch.autograd import Variable
from torch.optim.lr_scheduler import MultiStepLR
from torch.utils.data import Dataset
from torchvision import transforms
from tqdm.notebook import tqdm as tqdm


##**Model - Define ResNet Model**

In [3]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F

def Conv(in_channels, out_channels, kerner_size, stride, padding):
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, kerner_size, stride, padding, bias=False),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(True),
    )

class Classifier(nn.Module):
    def __init__(self, in_channel, out_channel):
        super(Classifier, self).__init__()
        self.body = nn.Sequential(
            nn.Linear(in_channel, out_channel),
            nn.Softmax(dim=0))
    def forward(self, x):
        return self.body(x)


class RandomBin(nn.Module):
    def __init__(self):
        super(RandomBin, self).__init__()
        self.conv1 = Conv(3, 16, 3, 1, 1)
        self.conv2 = Conv(16, 16, 3, 1, 1)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv3 = Conv(16, 32, 3, 1, 1)
        self.conv4 = Conv(32, 32, 3, 1, 1)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.conv5 = Conv(32, 64, 3, 1, 1)
        self.conv6 = Conv(64, 64, 3, 1, 1)
        self.pool3 = nn.MaxPool2d(2, 2)
        self.conv7 = Conv(64, 128, 3, 1, 1)
        self.conv8 = Conv(128, 128, 3, 1, 1)
        self.conv9 = Conv(128, 128, 3, 1, 1)
        self.pool5 = nn.MaxPool2d(2, 2)
        self.conv10 = Conv(128, 128, 3, 1, 1)
        self.conv11 = Conv(128, 128, 3, 1, 1)
        self.conv12 = Conv(128, 128, 3, 1, 1)
        self.HP = nn.Sequential(
            nn.MaxPool2d(2, 2),
            nn.AvgPool2d(kernel_size=7, stride=1)
        )


        self.fc1 = nn.Sequential(
            nn.Linear(128, 117),
            nn.Sigmoid()
        )

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                nn.init.xavier_normal_(m.weight)
                nn.init.constant_(m.bias, 0)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.pool1(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.pool2(x)
        x = self.conv5(x)
        x = self.conv6(x)
        x = self.pool3(x)
        x = self.conv7(x)
        x = self.conv8(x)
        x = self.conv9(x)
        x = self.pool5(x)
        x = self.conv10(x)
        x = self.conv11(x)
        x = self.conv12(x)
        x = self.HP(x)
        x = x.view((x.size(0), -1))

        x = self.fc1(x.view((x.size(0), -1)))
        
        x = F.normalize(x, p=1, dim=1)

        return x

    
import torch
import torch.nn as nn
import torch.nn.functional as F

def kl_loss(inputs, labels):
    criterion = nn.KLDivLoss(reduce=False)
    outputs = torch.log(inputs)
    loss = criterion(outputs, labels)
    loss = loss.sum()/loss.shape[0]
    return loss

def L1_loss(inputs, labels):
    criterion = nn.L1Loss(reduction='mean')
    loss = criterion(inputs, labels.float())
    return loss    

def normal_sampling(mean, label_k, std=2):
    return math.exp(-(label_k-mean)**2/(2*std**2))/(math.sqrt(2*math.pi)*std)

flip = transforms.RandomHorizontalFlip(p=1.)

##**Utils**

In [7]:
class AverageMeter(object):
    r"""Computes and stores the average and current value
    """
    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)


class ProgressMeter(object):
    def __init__(self, num_batches, *meters, prefix=""):
        self.batch_fmtstr = self._get_batch_fmtstr(num_batches)
        self.meters = meters
        self.prefix = prefix

    def print(self, batch):
        entries = [self.prefix + self.batch_fmtstr.format(batch)]
        entries += [str(meter) for meter in self.meters]
        print('\t'.join(entries))

    def _get_batch_fmtstr(self, num_batches):
        num_digits = len(str(num_batches // 1))
        fmt = '{:' + str(num_digits) + 'd}'
        return '[' + fmt + '/' + fmt.format(num_batches) + ']'


def accuracy(output, target, topk=(1,)):
    r"""Computes the accuracy over the $k$ top predictions for the specified values of k
    """
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        # _, pred = output.topk(maxk, 1, True, True)
        # pred = pred.t()
        # correct = pred.eq(target.view(1, -1).expand_as(pred))

        # faster topk (ref: https://github.com/pytorch/pytorch/issues/22812)
        _, idx = output.sort(descending=True)
        pred = idx[:,:maxk]
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

##**Parameter Settings**

In [8]:
model = 'resnet18' # resnet18, resnet50, resnet101
batch_size = 32  # Input batch size for training (default: 128)
epochs = 3 # Number of epochs to train (default: 100)
learning_rate = 1e-3 # Learning rate

seed = 0 # Random seed (default: 0)
num_classes = 1 ##regression
print_freq = 100
num_workers = 2
cuda = torch.cuda.is_available()
cudnn.benchmark = True  # Should make training should go faster for large models
train_val_ratio = 0.99

M = 30
N = 10

torch.manual_seed(seed)
if cuda:
    torch.cuda.manual_seed(seed)

##**Load and preprocess data**

In [9]:
class FacialDataset(Dataset):
    def __init__(self, data_path, transform):
        if not os.path.exists(data_path):
            raise Exception(f"[!] {self.data_path} not existed")
        self.imgs = []
        self.labels = []
        self.transform = transform
        self.age_path = sorted(glob(os.path.join(data_path, "*.*")))
        for pth in self.age_path:
            img = pth
            label = int(pth.split('_')[0].split('/')[-1])
            self.labels.append(label)
            self.imgs.append(img)
    def __getitem__(self, idx):
        image = self.transform(Image.open(self.imgs[idx]))
        age = self.labels[idx]
        label = [normal_sampling(int(age), i) for i in range(117)]
        label = [i if i > 1e-15 else 1e-15 for i in label]
        label = torch.Tensor(label)
        return image , label, age

    def __len__(self):
        return len(self.age_path)

def get_data_loader(data_path, batch_size, num_workers,train_val_ratio):
    train_transform = transforms.Compose([
        transforms.Resize((224,224)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomGrayscale(),
        transforms.RandomRotation(20),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5754, 0.4529, 0.3986],
                                    std=[0.2715, 0.2423, 0.2354])
    ])
    
    val_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5754, 0.4529, 0.3986],
                                    std=[0.2715, 0.2423, 0.2354])
    ])

    full_dataset = FacialDataset(data_path, val_transform)
    train_size = int(train_val_ratio * len(full_dataset))
    val_size = len(full_dataset) - train_size

    train_dataset, val_dataset = torch.utils.data.random_split(full_dataset, [train_size, val_size])
    train_dataset.dataset = FacialDataset(data_path, train_transform)
    
    torch.manual_seed(3334)
    train_loader = torch.utils.data.DataLoader(dataset=train_dataset,batch_size=batch_size, shuffle=True,num_workers=num_workers, pin_memory=False)
    val_loader = torch.utils.data.DataLoader(dataset=val_dataset,batch_size=batch_size,shuffle=False,num_workers=num_workers, pin_memory=True)
    return train_loader, val_loader

train_loader, val_loader = get_data_loader('./dataset/train',batch_size,num_workers,train_val_ratio)


In [10]:
def train(train_loader, epoch, model, optimizer):
    batch_time = AverageMeter('Time', ':6.3f')
    l2 = AverageMeter('L2 Loss', ':.4e')
    l1 = AverageMeter('L1 Loss', ':.4e')
    progress = ProgressMeter(len(train_loader), batch_time, l2,l1, prefix="Epoch: [{}]".format(epoch))
    # switch to train mode
    model.train()
    l1_criterion = kl_loss
    l2_criterion = L1_loss
    rank = torch.Tensor([i for i in range(117)]).cuda()
    end = time.time()
    for i, (input, label, age) in enumerate(train_loader):
        # measure data loading time
        age = age.float().cuda(gpu)
        input = input.cuda(gpu)
        label = label.cuda(gpu)
        # compute output
        output = model(input)
        ages = torch.sum(output*rank, dim=1)
#         print(output.size())
#         print(ages.size())
        l2_ = l2_criterion(ages, age)
        l1_ = l1_criterion(output, label)
        l2.update(l2_.item(), input.size(0))
        l1.update(l1_.item(), input.size(0))
        total_loss = l1_ + l2_
        # compute gradient and do SGD step
        optimizer.zero_grad()
        total_loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % print_freq == 0:
            progress.print(i)
            print(total_loss.item())
    # print('==> Train Accuracy: Loss {losses:.3f} || scores {r2_score:.4e}'.format(losses=losses, r2_score=r2))

def validation(val_loader,epoch, model):
    model.eval()
    with torch.no_grad():
        l2 = torch.nn.MSELoss().cuda(gpu)
        mse, num_samples = 0, 0
        rank = torch.Tensor([i for i in range(117)]).cuda()
        for i,(input,label,age) in enumerate(val_loader):
            flipped = flip(input).cuda(gpu)
            input = input.cuda(gpu)
            age = age.cuda(gpu)
            output = model(input)
            output_flipped = model(flipped)
            ages = torch.sum(output*rank, dim=1)
            ages_flipped = torch.sum(output_flipped*rank, dim=1)
            ages = ages/2 + ages_flipped/2
            l2_ = l2(age, ages)
            mse += torch.sum((ages - age)**2)
            num_samples += label.size(0)
        mse = mse.float() / num_samples
        RMSE = torch.sqrt(mse)
    print('==> Validate Accuracy:  L2 distance {:.3f} || RMSE {:.3f}'.format(l2_,RMSE))
    return RMSE


###########################################################
# model = mobilenetv3_small(num_classes=num_classes, width_mult=1.0).cuda(gpu)
# model = ResNet18(num_classes=num_classes).cuda(gpu)
# model = densenet(num_classes).cuda(gpu)
model = TinyAge().cuda(gpu)

# Check number of parameters your model
pytorch_total_params = sum(p.numel() for p in model.parameters())
print(f"Number of parameters: {pytorch_total_params}")
if int(pytorch_total_params) > 2000000:
    print('Your model has the number of parameters more than 2 millions..')
#     sys.exit()

# optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate,momentum=0.9, nesterov=True, weight_decay=5e-4)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
# optimizer = torch.optim.RMSprop(model.parameters(), lr=0.005, momentum=0.9, weight_decay=1e-5, eps=0.002)

# scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)
# scheduler = MultiStepLR(optimizer, milestones=[60, 90, 120], gamma=0.2)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 30, gamma=0.1)

best_acc = 1e5
for epoch in range(epochs):
    print("\n----- epoch: {}, lr: {} -----".format(
        epoch, optimizer.param_groups[0]["lr"]))

    # train for one epoch
    start_time = time.time()
    train(train_loader, epoch, model, optimizer)
    val_acc = validation(val_loader,epoch,model)

    elapsed_time = time.time() - start_time
    print('==> {:.2f} seconds to train this epoch\n'.format(elapsed_time))
    # learning rate scheduling
    scheduler.step()
    
    # Save model for best accuracy
    if best_acc > val_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), 'model_best.pt')

    torch.save(model.state_dict(),'model_latest.pt')
print(f"Best RMSE Accuracy: {best_acc}")

Number of parameters: 899941

----- epoch: 0, lr: 0.001 -----


/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))


Epoch: [0][  0/337]	Time  2.446 ( 2.446)	L2 Loss 2.9741e+01 (2.9741e+01)	L1 Loss 2.6563e+00 (2.6563e+00)
32.39742660522461
Epoch: [0][100/337]	Time  0.458 ( 0.369)	L2 Loss 9.9741e+00 (1.0538e+01)	L1 Loss 1.9555e+00 (2.0575e+00)
11.9296236038208
Epoch: [0][200/337]	Time  0.321 ( 0.359)	L2 Loss 9.2119e+00 (9.3043e+00)	L1 Loss 1.9247e+00 (1.9807e+00)
11.136592864990234
Epoch: [0][300/337]	Time  0.514 ( 0.355)	L2 Loss 6.4374e+00 (8.6830e+00)	L1 Loss 1.6661e+00 (1.9192e+00)
8.103525161743164
==> Validate Accuracy:  L2 distance 75.549 || RMSE 10.644
==> 124.93 seconds to train this epoch


----- epoch: 1, lr: 0.001 -----
Epoch: [1][  0/337]	Time  0.888 ( 0.888)	L2 Loss 7.4275e+00 (7.4275e+00)	L1 Loss 1.6460e+00 (1.6460e+00)
9.073555946350098
Epoch: [1][100/337]	Time  0.504 ( 0.368)	L2 Loss 6.0324e+00 (7.1136e+00)	L1 Loss 1.5894e+00 (1.6440e+00)
7.621860504150391
Epoch: [1][200/337]	Time  0.232 ( 0.361)	L2 Loss 7.0461e+00 (6.9945e+00)	L1 Loss 1.4768e+00 (1.6005e+00)
8.522857666015625
Epoch: [

## Make an evalutation csv file

This code makes an evaluation csv file for Competition submission.

**Don't change below code!!!**

In [6]:
class FacialDataset_test(Dataset):
    def __init__(self, data_path):
        if not os.path.exists(data_path):
            raise Exception(f"[!] {self.data_path} not existed")
        self.imgs = []
        self.transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5754, 0.4529, 0.3986],
                                    std=[0.2715, 0.2423, 0.2354])
        ])
        self.age_path = sorted(glob(os.path.join(data_path, "*.*")))
        for pth in self.age_path:
          # img = Image.open(pth)
          self.imgs.append(pth)
    def __getitem__(self, idx):
        image = self.transform(Image.open(self.imgs[idx]))
        return image

    def __len__(self):
        return len(self.age_path)

import torch
import pandas as pd
import argparse
import time
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader

rank = torch.Tensor([i for i in range(117)]).cuda()

def eval():
    test_dataset = FacialDataset_test("/content/dataset/test")
    test_loader = torch.utils.data.DataLoader(dataset=test_dataset,batch_size=1,shuffle=False)

    model = TinyAge().cuda()
    model.load_state_dict(torch.load('/content/model_best.pt'))
    model.eval()
    
    print('Make an evaluation csv file for submission...')
    Category = []
    for input in test_loader:
        input = input.cuda()
        output = model(input)
        output = [torch.sum(output*rank, dim=1).item()]
        # output = torch.argmax(output, dim=1)
        Category = Category + output
    print(Category)
    Id = list(range(0, len(test_loader)))
    samples = {
       'Id': Id,
       'Category': Category 
    }
    df = pd.DataFrame(samples, columns=['Id', 'Category'])

    df.to_csv('/content/submission_best.csv', index=False)
    print('Done!!')

    del model
    model = TinyAge().cuda()
    model.load_state_dict(torch.load('/content/model_latest.pt'))
    model.eval()
    
    print('Make an evaluation csv file for submission...')
    Category = []
    for input in test_loader:
        input = input.cuda()
        output = model(input)
        output = [torch.sum(output*rank, dim=1).item()]
        # output = torch.argmax(output, dim=1)
        Category = Category + output

    Id = list(range(0, len(test_loader)))
    samples = {
       'Id': Id,
       'Category': Category 
    }
    df = pd.DataFrame(samples, columns=['Id', 'Category'])

    df.to_csv('/content/submission_latest.csv', index=False)
    print('Done!!')

if __name__ == "__main__":
    eval()

Make an evaluation csv file for submission...
[28.60675811767578, 24.578821182250977, 45.798919677734375, 30.706905364990234, 30.057958602905273, 49.258148193359375, 18.98497772216797, 27.668563842773438, 1.3260090351104736, 28.261621475219727, 21.141321182250977, 50.419769287109375, 34.44218063354492, 22.319284439086914, 46.331172943115234, 21.44721221923828, 22.651086807250977, 40.337806701660156, 19.98101806640625, 1.9338594675064087, 29.36621856689453, 4.173193454742432, 34.29074478149414, 45.08757781982422, 1.4501423835754395, 19.314973831176758, 27.558067321777344, 33.13203048706055, 35.63618469238281, 48.93910217285156, 31.725353240966797, 31.984758377075195, 37.75635528564453, 2.378265619277954, 22.005281448364258, 51.0804328918457, 23.943708419799805, 43.042633056640625, 33.51108169555664, 20.78392219543457, 27.509992599487305, 27.931785583496094, 30.59229278564453, 28.899688720703125, 20.47808074951172, 32.6541748046875, 23.958078384399414, 21.23650550842285, 28.3424091339111